In [1]:
# Libraries
import mysql.connector
import pandas as pd
import numpy as np

In [2]:
# MYSQL Connection string

config = {
  'user': 'root',
  'password': 'Nls@123',
  'host': '127.0.0.1',
  'database': 'staging2',
  'auth_plugin' : 'mysql_native_password',
  'raise_on_warnings': True
}

In [3]:
# GL vs GL - NOSTRO16478501185683904.380004374953002
# One to one - NOSTRO164759560132575529500004372331001,NOSTRO164759504580810029650004366490003
# One to Many (OR) Many to One - NOSTRO1647589655262101109.620004366494004
# Many to Many - NOSTRO16480282679601478794.720004366490004

In [4]:
# Query
query = """
select his.AC_ENTRY_SR_NO As GL_ID,his.ACCOUNT_NO AS GL_ACCNO,his.VALUE_DT as GL_DATE,his.DRCR_IND as GL_CRDR,
CASE 
    WHEN his.AC_CCY = 'KES' then his.LCY_AMOUNT
    ELSE his.FCY_AMOUNT
    END as GL_AMOUNT,
ft.USER_REF_NUMBER as GL_REF,ft.SOURCE_REF_NO GL_SOURCE_REF,
CASE
    WHEN ft.PAYMENT_DETAILS IS NOT NULL AND ft.PAYMENT_DETAILS != '' THEN ft.PAYMENT_DETAILS
    WHEN ct.ADDL_TEXT IS NOT NULL THEN ct.ADDL_TEXT
    ELSE null
    END as GL_ADDL_TEXT,
his.AC_CCY as GL_CURRENCY
from statement$detail$history his 
left JOIN funds$transfer ft on ft.id = his.TRN_REF_NO
LEFT JOIN cstb$addl$text ct on ct.REFERENCE_NO = his.TRN_REF_NO
where his.AC_ENTRY_SR_NO in
(
select PRI_entryId from tbl_recon_nostro$matched$entry where fld_MatchType = 'MANUAL' and 
fld_UniqueReference ='NOSTRO16480282679601478794.720004366490004'
)
"""

In [5]:
# Fetch GL data from DB
mysql_connection = mysql.connector.connect(**config)
GL_df = pd.read_sql(query, con=mysql_connection)
mysql_connection.close()
GL_df.tail()

,GL_ID,GL_ACCNO,GL_DATE,GL_CRDR,GL_AMOUNT,GL_REF,GL_SOURCE_REF,GL_ADDL_TEXT,GL_CURRENCY
10,1175595887,0004366490004,2022-03-14,C,17482.12,None,None,None,EUR
11,1175595893,0004366490004,2022-03-14,D,91.22,None,None,None,EUR
12,1175595897,0004366490004,2022-03-14,D,18.24,None,None,None,EUR
13,1179466130,0004366490004,2022-03-22,D,400000.00,None,None,None,EUR
14,1179466158,0004366490004,2022-03-22,D,500000.00,None,None,None,EUR


In [6]:
# Fetch NOSTRO data from DB
query = """
select ID as RE_ID,ACCOUNT_NUMBER as RE_ACCNO,VALUE_DATE as RE_DATE,CRF_TYPE as RE_CRDR,
SWIFT_AMOUNT as RE_AMOUNT,ACC_OWNER_REF_FOR_DEBIT,ACC_SERVICING_INSTITUTE_REF_FOR_CREDIT from nostro$statement where id in  
(
select PRI_entryId from tbl_recon_nostro$matched$entry where 
fld_MatchType = 'MANUAL' and fld_UniqueReference ='NOSTRO16480282679601478794.720004366490004'
)
"""
mysql_connection = mysql.connector.connect(**config)
RE_df = pd.read_sql(query, con=mysql_connection)
mysql_connection.close()
RE_df.tail()

,RE_ID,RE_ACCNO,RE_DATE,RE_CRDR,RE_AMOUNT,ACC_OWNER_REF_FOR_DEBIT,ACC_SERVICING_INSTITUTE_REF_FOR_CREDIT
1,1261337,0004366490004,2022-03-22,CREDIT,500000.00,SBM 11943,220322051229
2,1261342,0004366490004,2022-03-22,DEBIT,-44707.99,940201633084 PAY,202203220087333
3,1261343,0004366490004,2022-03-22,DEBIT,-16226.15,940201633066 PAY,202203220088258
4,1261346,0004366490004,2022-03-22,DEBIT,-17338.43,940201633075 PAY,202203220088328
5,1261348,0004366490004,2022-03-22,CREDIT,500000.00,CAF0207543882/02,220322023131


In [7]:
# Converting time_stamp
GL_df['GL_DATE'] = pd.to_datetime(GL_df['GL_DATE'])
RE_df['RE_DATE'] = pd.to_datetime(RE_df['RE_DATE'])

# Remove negative signs
GL_df['GL_AMOUNT'] = GL_df['GL_AMOUNT'].abs()
# RE_df['RE_AMOUNT'] = RE_df['RE_AMOUNT'].abs()

# Adding negative sign to Debit transactions
GL_df.loc[(GL_df['GL_CRDR'] == 'D'), 'GL_AMOUNT'] =  GL_df['GL_AMOUNT']*-1
# RE_df.loc[(RE_df['RE_CRDR'] == 'D'), 'RE_AMOUNT'] =  RE_df['RE_AMOUNT']*-1

# Replace CRDR value
RE_df['RE_CRDR'] = RE_df['RE_CRDR'].replace(['CREDIT', 'DEBIT'], ['C','D'])
RE_df.head()

,RE_ID,RE_ACCNO,RE_DATE,RE_CRDR,RE_AMOUNT,ACC_OWNER_REF_FOR_DEBIT,ACC_SERVICING_INSTITUTE_REF_FOR_CREDIT
0,1261336,0004366490004,2022-03-22,C,400000.00,SBICNX2775SBMKNA,220322047219
1,1261337,0004366490004,2022-03-22,C,500000.00,SBM 11943,220322051229
2,1261342,0004366490004,2022-03-22,D,-44707.99,940201633084 PAY,202203220087333
3,1261343,0004366490004,2022-03-22,D,-16226.15,940201633066 PAY,202203220088258
4,1261346,0004366490004,2022-03-22,D,-17338.43,940201633075 PAY,202203220088328


In [8]:
# Define variance amount
Variance = 1
# Creating ismatched,match table and match id column
GL_df['ISMATCHED'] = 0
RE_df['ISMATCHED'] = 0
GL_df['MATCH_TABLE'] = np.nan
GL_df['MATCH_ID'] = ''
# 
# Dataframe info
GL_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   GL_ID          15 non-null     object        
 1   GL_ACCNO       15 non-null     object        
 2   GL_DATE        15 non-null     datetime64[ns]
 3   GL_CRDR        15 non-null     object        
 4   GL_AMOUNT      15 non-null     float64       
 5   GL_REF         3 non-null      object        
 6   GL_SOURCE_REF  3 non-null      object        
 7   GL_ADDL_TEXT   3 non-null      object        
 8   GL_CURRENCY    15 non-null     object        
 9   ISMATCHED      15 non-null     int64         
 10  MATCH_TABLE    0 non-null      float64       
 11  MATCH_ID       15 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 1.5+ KB


In [9]:
# With in GL account match
for i in range(0,len(GL_df)-1):
    if (GL_df['ISMATCHED'][i] == 1):
        continue
    for j in range(i+1,len(GL_df)):
        if((GL_df['ISMATCHED'][j] == 1)):
            continue
        amount = abs(GL_df['GL_AMOUNT'][i] + GL_df['GL_AMOUNT'][j])
        if amount <= Variance:
            GL_df.loc[i,'ISMATCHED'] = 1
            GL_df.loc[j,'ISMATCHED'] = 1
            GL_df.loc[j,'MATCH_TABLE'] = 'statement$detail$history'
            GL_df.loc[j,'MATCH_ID'] = str(GL_df['GL_ID'][i])
            break

In [10]:
# One to one match with GL to Nostro to avoid more combinations
for i in range(0,len(GL_df)):
    if (GL_df['ISMATCHED'][i] == 1):
        continue
    for j in range(0,len(RE_df)):
        if((RE_df['ISMATCHED'][j] == 1)):
            continue
        amount = abs(GL_df['GL_AMOUNT'][i] + RE_df['RE_AMOUNT'][j])
        if amount <= Variance:
            GL_df.loc[i,'ISMATCHED'] = 1
            RE_df.loc[j,'ISMATCHED'] = 1
            GL_df.loc[i,'MATCH_TABLE'] = 'tbl_recon_nostro$matched$entry'
            GL_df.loc[i,'MATCH_ID'] = str(RE_df['RE_ID'][j])
            break

In [11]:
# # Merging 2 dataframes
# df = pd.merge(GL_df, RE_df, left_on=['GL_ACCNO','GL_AMOUNT'],right_on=['RE_ACCNO','RE_AMOUNT'])

# # Dropping duplicate values
# df = df.drop_duplicates(["GL_ID","RE_ID"],ignore_index = True)
# df.head(20)

In [12]:
# # Verifing GL credit or debit not equals to NOSTRO credit or debit
# df = df.query("GL_CRDR != RE_CRDR")
# df.head(10)

In [13]:
# # Converting trans reference as list
# GL_ID = df['GL_ID'].unique().tolist()
# RE_ID = df['RE_ID'].unique().tolist()

In [14]:
# # Filtering and removing the existing matches as above (GL)
# GL_df = GL_df[~GL_df['GL_ID'].isin(GL_ID)]
# GL_df.reset_index(drop=True, inplace=True)
# GL_df.tail()

In [15]:
# # Filtering and removing the existing matches as above (NOSTRO)
# RE_df = RE_df[~RE_df['RE_ID'].isin(RE_ID)]
# RE_df.reset_index(drop=True, inplace=True)
# RE_df.tail()

In [16]:
# Posiblities

# GL to Nostro and Nostro to GL
#    One to One
#    One to many
#    Many to one
#    many to many

# multiple credit,debit sums alowed in both ends and the variance should below defined variance
# once the match is found I should not consider it again
# date should be match for bulk matching and can allow diffrent dates for one to one match

# With in same GL account
#    One to One

# With in same Nostro account
#    Nil

# Allowed Maximum Variance
#    MAX = 1

In [17]:
from itertools import combinations

In [18]:
# dict_data = GL_df.set_index('GL_ID')['GL_AMOUNT'].to_dict()

# length = len(dict_data)
# GL_per_com = [i for j in range(length + 1) for i in combinations(dict_data, j) if sum(map(dict_data.get, i))]

# #print(GL_per_com)
# len(GL_per_com)

In [19]:
dict_data = GL_df.loc[GL_df['ISMATCHED']==0].set_index('GL_ID')['GL_AMOUNT'].to_dict()
length = len(dict_data)
GL_per_com = [i for j in range(length + 1) for i in combinations(dict_data, j) if sum(map(dict_data.get, i))]

#print(GL_per_com)
len(GL_per_com)

4095

In [20]:
Max_per_com = 5
GL_per_com_list = []
for i in GL_per_com:
    if len(i) <= Max_per_com:
        GL_per_com_list.append(i) 

print(GL_per_com_list)
# len(GL_per_com_list)

[('1173020685',), ('1172625172',), ('1172625178',), ('1172625182',), ('1175599195',), ('1172611206',), ('1172611212',), ('1172611216',), ('1173021737',), ('1175595887',), ('1175595893',), ('1175595897',), ('1173020685', '1172625172'), ('1173020685', '1172625178'), ('1173020685', '1172625182'), ('1173020685', '1175599195'), ('1173020685', '1172611206'), ('1173020685', '1172611212'), ('1173020685', '1172611216'), ('1173020685', '1173021737'), ('1173020685', '1175595887'), ('1173020685', '1175595893'), ('1173020685', '1175595897'), ('1172625172', '1172625178'), ('1172625172', '1172625182'), ('1172625172', '1175599195'), ('1172625172', '1172611206'), ('1172625172', '1172611212'), ('1172625172', '1172611216'), ('1172625172', '1173021737'), ('1172625172', '1175595887'), ('1172625172', '1175595893'), ('1172625172', '1175595897'), ('1172625178', '1172625182'), ('1172625178', '1175599195'), ('1172625178', '1172611206'), ('1172625178', '1172611212'), ('1172625178', '1172611216'), ('1172625178', 

In [21]:
dict_data = RE_df.loc[RE_df['ISMATCHED']==0].set_index('RE_ID')['RE_AMOUNT'].to_dict()
length = len(dict_data)
RE_per_com = [i for j in range(length +  1) for i in combinations(dict_data, j) if sum(map(dict_data.get, i))]

#print(RE_per_com)
len(RE_per_com)

7

In [22]:
RE_per_com_list = []
for i in RE_per_com:
    if len(i) <= Max_per_com:
        RE_per_com_list.append(i) 

print(RE_per_com_list)
# len(RE_per_com_list)

[(1261342,), (1261343,), (1261346,), (1261342, 1261343), (1261342, 1261346), (1261343, 1261346), (1261342, 1261343, 1261346)]


In [23]:
# filter the data where ID in above using nested for loop
# sum of all equals to another sum of all or with in the variance 
# map the respective id

In [24]:
for i in range(0,len(GL_per_com_list)):
    if 1 in (GL_df[GL_df['GL_ID'].isin(list(GL_per_com_list[i]))]['ISMATCHED'].values):
        continue
    if len(GL_df[GL_df['GL_ID'].isin(list(GL_per_com_list[i]))]['GL_DATE'].dt.strftime('%m/%Y').unique()) > 1:
        continue
    for j in range(0,len(RE_per_com_list)):
        if 1 in (RE_df[RE_df['RE_ID'].isin(list(RE_per_com_list[j]))]['ISMATCHED'].values):
            continue
        if len(RE_df[RE_df['RE_ID'].isin(list(RE_per_com_list[j]))]['RE_DATE'].dt.strftime('%m/%Y').unique()) > 1:
            continue
        if ((GL_df[GL_df['GL_ID'].isin(list(GL_per_com_list[i]))]['GL_DATE'].dt.strftime('%m/%Y').unique()[0]) != (RE_df[RE_df['RE_ID'].isin(list(RE_per_com_list[j]))]['RE_DATE'].dt.strftime('%m/%Y').unique()[0])):
            continue
        amount = abs((GL_df[GL_df['GL_ID'].isin(list(GL_per_com_list[i]))]['GL_AMOUNT'].sum()) + (RE_df[RE_df['RE_ID'].isin(list(RE_per_com_list[j]))]['RE_AMOUNT'].sum()))
        if amount <= Variance:
            GL_df.loc[GL_df['GL_ID'].isin(list(GL_per_com_list[i])),'ISMATCHED'] = 1
            RE_df.loc[RE_df['RE_ID'].isin(list(RE_per_com_list[j])),'ISMATCHED'] = 1
            GL_df.loc[GL_df['GL_ID'].isin(list(GL_per_com_list[i])),'MATCH_TABLE'] = 'tbl_recon_nostro$matched$entry'
            GL_df.loc[GL_df['GL_ID'].isin(list(GL_per_com_list[i])),'MATCH_ID'] = str(list(RE_per_com_list[j]))
            break

In [25]:
df = GL_df[(GL_df['ISMATCHED'] == 1) & (GL_df['MATCH_TABLE'].notnull())]
df

,GL_ID,GL_ACCNO,GL_DATE,GL_CRDR,GL_AMOUNT,GL_REF,GL_SOURCE_REF,GL_ADDL_TEXT,GL_CURRENCY,ISMATCHED,MATCH_TABLE,MATCH_ID
0,1173020685,0004366490004,2022-03-09,D,-124.35,000INAC220680001,000INAC220680001,//008UILC212950004 REFUND OF,EUR,1,tbl_recon_nostro$matched$entry,[1261343]
1,1179466146,0004366490004,2022-03-22,D,-500000.00,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,1261337
2,1172625172,0004366490004,2022-03-08,C,44852.16,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261342]
3,1172625178,0004366490004,2022-03-08,D,-91.62,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261343]
4,1172625182,0004366490004,2022-03-08,D,-18.32,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261343]
5,1175599195,0004366490004,2022-03-14,D,-34.23,000INAC220730008,000INAC220730008,"//008UILC212950002 AMD, SWIFT AND",EUR,1,tbl_recon_nostro$matched$entry,[1261342]
6,1172611206,0004366490004,2022-03-08,C,16460.44,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261343]
7,1172611212,0004366490004,2022-03-08,D,-91.62,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261342]
8,1172611216,0004366490004,2022-03-08,D,-18.32,None,None,None,EUR,1,tbl_recon_nostro$matched$entry,[1261342]
9,1173021737,0004366490004,2022-03-09,D,-34.23,000INAC220680002,000INAC220680002,//008UILC212950001 REFUND OF,EUR,1,tbl_recon_nostro$matched$entry,[1261346]


In [26]:
df = df.groupby(['MATCH_ID','MATCH_TABLE'])['GL_ID'].apply(list).reset_index(name='STATEMENT_ID')
df = df[['STATEMENT_ID', 'MATCH_TABLE', 'MATCH_ID']]

for i in range (0,len(df)):
    if(type(eval(df['MATCH_ID'][i])) == list):
        continue
    df.loc[i,'MATCH_ID'] = str([int(df['MATCH_ID'][i])])
df

,STATEMENT_ID,MATCH_TABLE,MATCH_ID
0,[1179466130],tbl_recon_nostro$matched$entry,[1261336]
1,[1179466146],tbl_recon_nostro$matched$entry,[1261337]
2,[1179466158],tbl_recon_nostro$matched$entry,[1261348]
3,"[1172625172, 1175599195, 1172611212, 1172611216]",tbl_recon_nostro$matched$entry,[1261342]
4,"[1173020685, 1172625178, 1172625182, 1172611206]",tbl_recon_nostro$matched$entry,[1261343]
5,"[1173021737, 1175595887, 1175595893, 1175595897]",tbl_recon_nostro$matched$entry,[1261346]


In [27]:
json_data = df.to_json(orient='records')[1:-1].replace('},{', '} {')
json_data

'{"STATEMENT_ID":["1179466130"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261336]"} {"STATEMENT_ID":["1179466146"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261337]"} {"STATEMENT_ID":["1179466158"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261348]"} {"STATEMENT_ID":["1172625172","1175599195","1172611212","1172611216"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261342]"} {"STATEMENT_ID":["1173020685","1172625178","1172625182","1172611206"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261343]"} {"STATEMENT_ID":["1173021737","1175595887","1175595893","1175595897"],"MATCH_TABLE":"tbl_recon_nostro$matched$entry","MATCH_ID":"[1261346]"}'

In [37]:
Acc_No = '0004374954001'
# '0004366490004'
    
query = """
select his.AC_ENTRY_SR_NO As GL_ID,his.ACCOUNT_NO AS GL_ACCNO,his.VALUE_DT as GL_DATE,his.DRCR_IND as GL_CRDR,
CASE 
    WHEN his.AC_CCY = 'KES' then his.LCY_AMOUNT
    ELSE his.FCY_AMOUNT
    END as GL_AMOUNT,
ft.USER_REF_NUMBER as GL_REF,ft.SOURCE_REF_NO GL_SOURCE_REF,
CASE
    WHEN ft.PAYMENT_DETAILS IS NOT NULL AND ft.PAYMENT_DETAILS != '' THEN ft.PAYMENT_DETAILS
    WHEN ct.ADDL_TEXT IS NOT NULL THEN ct.ADDL_TEXT
    ELSE null
    END as GL_ADDL_TEXT,
his.AC_CCY as GL_CURRENCY
from statement$detail$history his 
left JOIN funds$transfer ft on ft.id = his.TRN_REF_NO
LEFT JOIN cstb$addl$text ct on ct.REFERENCE_NO = his.TRN_REF_NO
where his.AC_ENTRY_SR_NO in
(
select PRI_entryId from tbl_recon_nostro$unmatched$entry where fld_account_Number = """+Acc_No+"""
)
"""

In [38]:
# Fetch GL data from DB
mysql_connection = mysql.connector.connect(**config)
GL_df = pd.read_sql(query, con=mysql_connection)
mysql_connection.close()
GL_df.tail()

,GL_ID,GL_ACCNO,GL_DATE,GL_CRDR,GL_AMOUNT,GL_REF,GL_SOURCE_REF,GL_ADDL_TEXT,GL_CURRENCY
0,1278786183,0004374954001,2022-10-21,C,5476.29,FIN211022TT001,FIN211022TT001,//CCM MONTHLY SERVICE,USD
1,1279142716,0004374954001,2022-10-20,C,1136390.00,None,None,None,USD
2,1278620779,0004374954001,2022-10-19,D,39460.90,None,None,None,USD
3,1278800736,0004374954001,2022-10-19,C,8000.00,I221019174930612,I221019174930612,None,USD
4,1278684151,0004374954001,2022-10-19,D,613.88,None,None,"ASUS GLOBAL PTE LTD15A,CHANGI BUSINESS PARK CE...",USD


In [39]:
query = """
select ID as RE_ID,ACCOUNT_NUMBER as RE_ACCNO,VALUE_DATE as RE_DATE,CRF_TYPE as RE_CRDR,
SWIFT_AMOUNT as RE_AMOUNT,ACC_OWNER_REF_FOR_DEBIT,ACC_SERVICING_INSTITUTE_REF_FOR_CREDIT from nostro$statement where id in
(
select PRI_entryId from tbl_recon_nostro$unmatched$entry where fld_account_Number = """+Acc_No+"""
)
"""

In [40]:
mysql_connection = mysql.connector.connect(**config)
RE_df = pd.read_sql(query, con=mysql_connection)
mysql_connection.close()
RE_df.tail()

,RE_ID,RE_ACCNO,RE_DATE,RE_CRDR,RE_AMOUNT,ACC_OWNER_REF_FOR_DEBIT,ACC_SERVICING_INSTITUTE_REF_FOR_CREDIT
20,1539891,0004374954001,2022-10-18,DEBIT,-2642.04,22AHM353760O1M57,F012291004F602
21,1540250,0004374954001,2022-10-19,CREDIT,39460.91,262233-2,C0022920668901
22,1540260,0004374954001,2022-10-19,DEBIT,-2519.59,22AIM25055VO1K69,F0122920044C02
23,1540304,0004374954001,2022-10-20,CREDIT,166.58,TT06090220124521,S062293055E901
24,1540323,0004374954001,2022-10-20,DEBIT,-1964.48,22AJM2540ABO1N64,F0122930132402
